In [1]:
import numpy as np 
import pandas as pd 
import netCDF4
import pickle 
import os
import pandas as pd
import h5py 
import numpy as np

def Reshape( X ):
    x =[ ]
    for i in range(X.shape[1]):
        x.append(X[: , i  , :])
    return np.array(x)

def rescale( data  ,Ref ) :
    
    means = np.mean(Ref , axis = 0 )
    std = np.std( Ref , axis =0)
    return (data -means ) / std

def Dict_to_hdf(Dict , of ):
    HDF = h5py.File(of, 'w')
    for k in Dict :    
        test = HDF.create_dataset( k , data = Dict[k] )
    HDF.close()   


def nc_to_pd(nc):
    try:
        Keys = list(nc.keys())
    except:
        Keys = list(nc.variables)
    Panda ={ }
    for k in Keys:
        Panda[k]= np.array(nc[k] )    
    return pd.DataFrame(Panda)

def DF_bites_to_str(DF):
        
    for k in DF :
        if type(DF[k][0])  == np.bytes_ :
            DF[k]  = np.array( DF[k]).astype(str)
    return DF


In [2]:

Folder = "D:/Projects/precipitation_AUS/dataset/0024/"
files = os.listdir(Folder)
hdfs = []

for file in files :
    hdf = h5py.File(Folder + file , 'r+')
    hdfs.append( hdf)



In [3]:
list(hdfs[1].keys())

['CNN_RA_(3,3)',
 'CNN_RA_(5,5)',
 'CNN_RA_(7,7)',
 'CNN_RA_C_(3,3)',
 'CNN_RA_C_(5,5)',
 'CNN_RA_C_(7,7)',
 'CNN_RA_C_[-1](7,7)',
 'CNN_RA_C_[-2](7,7)',
 'CNN_RA_C_[-3](7,7)',
 'DAY',
 'LATITUDE',
 'LONGITUDE',
 'LSTM_RA_3(-4)',
 'MONTH',
 'NAME',
 'PRCP',
 'YEAR']

In [4]:
needed_keys = [ 'NAME'  , 'LATITUDE' , 'LONGITUDE' ,  
               'YEAR'  , 'MONTH'   ,  'DAY'  , 'PRCP'  
               , 'CAL_P' , 'ERA5_t2m' , 'ERA5_tcw'] 

DF = pd.DataFrame()
for k in needed_keys :
    DF[k] = np.array(hdfs[0][k])

    

In [5]:
from keras import layers
from keras.layers import Dense, Flatten
from keras.models import Sequential
from sklearn.ensemble import RandomForestRegressor

class RoundTable: 
    def __init__(self   , cnn  , shape_cnn , lstm , shape_lstm  , rf ):
        self.lstm =  lstm 

        self.cnn = cnn

        self.rf = rf

        self.CNN  =  Sequential()  
        self.shape_cnn = shape_cnn
        self.shape_lstm  = shape_lstm
        
        self.CNN.add(layers.Conv1D(cnn['layers'][0], kernel_size= cnn['kernels'][0] ,
                                    activation=cnn['activation_function'][0] , input_shape=  (  shape_cnn[1], shape_cnn[2] )))

        for i in range(1,len(cnn['layers'])):
            self.CNN .add(layers.Conv1D(cnn['layers'][i], kernel_size= cnn['kernels'][i] , activation=cnn['activation_function'][i] ))

        self.CNN.add(Flatten())

        self.CNN .add(Dense(lstm['words']))
        self.CNN .compile(loss=cnn['loss'], optimizer= cnn['Optimizer'])

        self.LSTM =  Sequential()  
                
        self.LSTM  = Sequential()
        self.LSTM.add(layers.LSTM(lstm['layers'][0] ,return_sequences=True, activation= lstm['activation_function'][0],
                        input_shape= (shape_lstm[1]  , shape_lstm[2]   ))) # input_shape=( x_train.shape[1]  , 1)
        
        for i in range(len(lstm['layers'])-1) :
            self.LSTM.add(layers.LSTM(lstm['layers'][i+1],activation=lstm['activation_function'][i+1], return_sequences=True))
        self.LSTM.add(Flatten())
        self.LSTM.add(Dense(lstm['words']))
        self.LSTM.compile(loss= lstm['loss']  , optimizer=lstm['Optimizer'] )
            
        self.RF = RandomForestRegressor( n_estimators =  rf['number_of_trees'] ,
                                max_depth = rf[ 'depth'] , 
                                min_samples_leaf= rf[ 'min_samples_leaf']   , 
                                random_state= rf[ 'random_state']  ,
                                n_jobs =rf[ 'n_jobs']    ,
                                verbose= 1)

    def fit_members(self  , x_cnn , x_lstm  , obs_train ):
        self.CNN.fit( x_cnn , obs_train, batch_size=self.cnn['size_batch'],epochs=self.cnn['n_epoch'], verbose=1)
        self.LSTM.fit(x_lstm, obs_train, epochs=self.lstm['n_epoch'], batch_size = self.lstm['size_batch'], verbose=1)
        self.CNN_predict = self.CNN.predict( x_cnn)
        self.LSTM_predict = self.LSTM.predict( x_lstm)

    def concat_opinions(self) :
        self.opn =   np.concatenate( [self.CNN_predict  , self.LSTM_predict ] , axis= 1   )

    def fit_decision(self , obs_train ):
        self.RF.fit(self.opn  , obs_train )

    def prediction(self , x_cnn , x_lstm):
        self.CNN_predict = self.CNN.predict( x_cnn)
        self.LSTM_predict = self.LSTM.predict( x_lstm)
        self.opnions =   np.concatenate( [self.CNN_predict  , self.LSTM_predict ] , axis= 1   )
        return self.RF.predict(self.opnions )






In [7]:

def read_for(  DF , hdfs , ord_Keys  , obs_key , Rescale   ):
    
    for Keys in ord_Keys:
        for k in Keys:
            for hdf in hdfs :
                if k in list(hdf.keys()):
                    DF[k] = np.array(hdf[k])


    train = DF[DF['YEAR']  <= 2020  ]
    test = DF[DF['YEAR']  > 2020]

    df = {}

    df['train']  , df['test']  =  train , test


    X = {}

    X_train =[]
    X_test =[]

    X_DF = []

    for K in ord_Keys : 

        X['train'] = df['train'][ K  ].values
        X['test'] = df['test'][ K  ].values
        X_train.append(rescale(  X['train'] , X['train']  )) 
        X_test.append( rescale(  X['test'] , X['train']  ) )

        X_DF.append(rescale(   DF[ K  ].values , X['train']  ))

    obs_train  = np.array(df['train'][obs_key])
    obs_test = np.array(df['test'][obs_key])

    X_train = np.array(X_train)
    X_test = np.array(X_test)

    return {  'X_train':  X_train  ,'X_test' : X_test  ,'obs_train' :  obs_train  , 'obs_test' : obs_test  , 'X_DF' : X_DF}





In [8]:

kk = [  'CAL_P'  ,    'PDIR'   ,   'ERA5_tp'  , 'ERA5_cape' , 'ERA5_t2m'  ,  'ERA5_tcw'] 

cnn_keys = [  kk    , 
          [k_ +'_ave_box(3, 3)'  for k_ in kk] ,
          [k_ +'_std_box(3, 3)'   for k_ in kk]  ,
          [k_ +'_Ci_box(3, 3)'   for k_ in kk]  , 
          [k_ +'_Cj_box(3, 3)'   for k_ in kk] ,
          
          [k_ +'_ave_box(5, 5)'  for k_ in kk] ,
        [k_ +'_std_box(5, 5)'   for k_ in kk]  , 
        [k_ +'_Ci_box(5, 5)'   for k_ in kk]  , 
        [k_ +'_Cj_box(5, 5)'   for k_ in kk] ,
                
        [k_ +'_ave_box(7, 7)'  for k_ in kk] ,
        [k_ +'_std_box(7, 7)'   for k_ in kk]  , 
        [k_ +'_Ci_box(7, 7)'   for k_ in kk]  , 
        [k_ +'_Cj_box(7, 7)'   for k_ in kk] ,


        [k_ +'(-2)_ave_box(3, 3)'  for k_ in kk] ,
        [k_ +'(-2)_std_box(3, 3)'   for k_ in kk]  , 
        [k_ +'(-2)_Ci_box(3, 3)'   for k_ in kk]  , 
        [k_ +'(-2)_Cj_box(3, 3)'   for k_ in kk]  , 

        [k_ +'(-2)_ave_box(5, 5)'  for k_ in kk] ,
        [k_ +'(-2)_std_box(5, 5)'   for k_ in kk]  , 
        [k_ +'(-2)_Ci_box(5, 5)'   for k_ in kk]  , 
        [k_ +'(-2)_Cj_box(5, 5)'   for k_ in kk] ,

        [k_ +'(-2)_ave_box(7, 7)'  for k_ in kk] ,
        [k_ +'(-2)_std_box(7, 7)'   for k_ in kk]  , 
        [k_ +'(-2)_Ci_box(7, 7)'   for k_ in kk]  , 
        [k_ +'(-2)_Cj_box(7, 7)'   for k_ in kk] ,
        

        [k_ +'(-3)_ave_box(3, 3)'  for k_ in kk] ,
        [k_ +'(-3)_std_box(3, 3)'   for k_ in kk]  , 
        [k_ +'(-3)_Ci_box(3, 3)'   for k_ in kk]  , 
        [k_ +'(-3)_Cj_box(3, 3)'   for k_ in kk]  , 

        [k_ +'(-3)_ave_box(5, 5)'  for k_ in kk] ,
        [k_ +'(-3)_std_box(5, 5)'   for k_ in kk]  , 
        [k_ +'(-3)_Ci_box(5, 5)'   for k_ in kk]  , 
        [k_ +'(-3)_Cj_box(5, 5)'   for k_ in kk] ,

        [k_ +'(-3)_ave_box(7, 7)'  for k_ in kk] ,
        [k_ +'(-3)_std_box(7, 7)'   for k_ in kk]  , 
        [k_ +'(-3)_Ci_box(7, 7)'   for k_ in kk]  , 
        [k_ +'(-3)_Cj_box(7, 7)'   for k_ in kk] ,
        

        [k_ +'(-1)_ave_box(3, 3)'  for k_ in kk] ,
        [k_ +'(-1)_std_box(3, 3)'   for k_ in kk]  , 
        [k_ +'(-1)_Ci_box(3, 3)'   for k_ in kk]  , 
        [k_ +'(-1)_Cj_box(3, 3)'   for k_ in kk]  , 

        [k_ +'(-1)_ave_box(5, 5)'  for k_ in kk] ,
        [k_ +'(-1)_std_box(5, 5)'   for k_ in kk]  , 
        [k_ +'(-1)_Ci_box(5, 5)'   for k_ in kk]  , 
        [k_ +'(-1)_Cj_box(5, 5)'   for k_ in kk] ,

        [k_ +'(-1)_ave_box(7, 7)'  for k_ in kk] ,
        [k_ +'(-1)_std_box(7, 7)'   for k_ in kk]  , 
        [k_ +'(-1)_Ci_box(7, 7)'   for k_ in kk]  , 
        [k_ +'(-1)_Cj_box(7, 7)'   for k_ in kk] 

            ]

lstm_keys = [ kk , 
   [  'CAL_P(-1)'  ,    'PDIR(-1)'   ,   'ERA5_tp(-1)'     ,   'ERA5_cape(-1)'    , 'ERA5_t2m(-1)' ,  'ERA5_tcw(-1)'  ]  ,
   [  'CAL_P(-2)'  ,    'PDIR(-2)'   ,   'ERA5_tp(-2)'     ,   'ERA5_cape(-2)'    , 'ERA5_t2m(-2)' ,  'ERA5_tcw(-2)'  ]  ,
  [  'CAL_P(-3)'  ,    'PDIR(-3)'   ,   'ERA5_tp(-3)'     ,   'ERA5_cape(-3)'    , 'ERA5_t2m(-3)' ,  'ERA5_tcw(-3)'  ]]


CNN_data = read_for(  DF, hdfs , cnn_keys  , 'PRCP' , True   )

LSTM_data = read_for(  DF, hdfs , lstm_keys  , 'PRCP' , True   )





C:\Users\S4055367\AppData\Local\Temp\ipykernel_14344\2242962215.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  DF[k] = np.array(hdf[k])
C:\Users\S4055367\AppData\Local\Temp\ipykernel_14344\2242962215.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  DF[k] = np.array(hdf[k])
C:\Users\S4055367\AppData\Local\Temp\ipykernel_14344\2242962215.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at onc

In [9]:
CNN_data['x_train'] = Reshape(CNN_data['X_train'] ) 
CNN_data['x_test'] = Reshape(CNN_data['X_test'] ) 

LSTM_data['x_train'] = Reshape(LSTM_data['X_train'] ) 
LSTM_data['x_test'] = Reshape(LSTM_data['X_test'] ) 


In [33]:

RF = {
        'type' : 'RF', 
        'Rescale' : True ,
        'number_of_trees' :  20,
        'depth' : 5,
        'min_samples_leaf' : 3  ,
        'random_state'  : 1,
        'n_jobs'  : 5, 
        }

CNN =  { 
        'words' : 4 , 
    'layers': [64]*5,
    'activation_function'  : ['relu']*5 ,
    'kernels' : [ 10  , 8  , 6 , 4  , 2 ,2  ],
    'n_epoch': 10,
    'size_batch' : 500,
    'Optimizer' : 'adam' ,
    'loss'  : 'mse'
}


LSTM = {   
    'words' : 4 , 
    'layers': [64]*3,
    'activation_function'  : ['relu'] *3,
    'n_epoch': 10 ,
    'size_batch' : 500,
    'Optimizer' : 'adam',
    'loss'  : 'mse'
 }

In [34]:
RT  = RoundTable( cnn  = CNN  , 
                 shape_cnn = CNN_data['x_train'].shape , 
                 lstm  = LSTM , shape_lstm =LSTM_data['x_train'].shape    ,
                   rf  =RF  )

c:\Users\S4055367\AppData\Local\anaconda3\envs\ML\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\S4055367\AppData\Local\anaconda3\envs\ML\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [147]:
obs_train = np.array( [LSTM_data['obs_train'] for i in range(8)  ] )
obs_test = np.array( [LSTM_data['obs_test'] for i in range(8)  ] )

In [12]:
from math import  * 

In [25]:
def Sgn(x , th):
    return ((x-th + 0.0001) / np.absolute(x -th + 0.0001) +1) /2

In [29]:
obs_train = np.array( [LSTM_data['obs_train'] ,  np.log(LSTM_data['obs_train']  + e ) , Sgn(LSTM_data['obs_train']  , 60 ) , np.tanh(LSTM_data['obs_train']) ]   )


In [51]:
obs_train= rescale(obs_train, obs_train)

In [53]:
RT.fit_members( x_cnn  = CNN_data['x_train'] , x_lstm = LSTM_data['x_train']  ,  obs_train=   obs_train.T )

Epoch 1/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 87s 43ms/step - loss: 0.3545
Epoch 2/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 99s 49ms/step - loss: 0.1336
Epoch 3/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 92s 46ms/step - loss: 0.1293
Epoch 4/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 83s 41ms/step - loss: 0.1251
Epoch 5/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 108s 53ms/step - loss: 0.1225
Epoch 6/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 80s 39ms/step - loss: 0.1211
Epoch 7/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 130s 64ms/step - loss: 0.1191
Epoch 8/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 88s 43ms/step - loss: 0.1180
Epoch 9/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 128s 63ms/step - loss: 0.1167
Epoch 10/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 116s 57ms/step - loss: 0.1156
Epoch 1/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 66s 33ms/step - loss: 0.3082
Epoch 2/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 48s 24ms/step - loss: 0.1285
Epoch 3/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 53s 26ms/step - loss: 0.1275
Epoch 4/10
2028/2028 ━━━━━━━━━━━━━━━━━━━━ 96s 47ms/step - loss: 0.1272
E

In [54]:
RT.concat_opinions()

In [55]:
RT.fit_decision(LSTM_data['obs_train'])

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  20 out of  20 | elapsed:   21.6s finished


In [56]:
predicted =  RT.prediction( CNN_data['x_test']  , LSTM_data['x_test']   )

15569/15569 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step
15569/15569 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step


[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  20 out of  20 | elapsed:    0.0s finished


In [39]:
import evaluation_tools


In [57]:
evaluation_tools.evaluation_criteria( np.array(LSTM_data['obs_test'] )  ,  predicted )

{'r': 0.6859996440110422,
 'NSE': 0.4701935340128103,
 'RMSE': 5.978162038857663,
 'MAE': 1.7448432177229285,
 'MBE': -0.08508561897704989,
 'PBIAS': -3.9823395083310458,
 'KGE': 0.5419029062720656,
 'spearman': 0.6867035970196567,
 'std': 5.493226345279872,
 'obs_std': 8.213136775672716,
 'r_heavy': 0.7976069197885983,
 'NSE_heavy': 0.5514379238546709,
 'RMSE_heavy': 35.22149475418087,
 'MAE_heavy': 26.54386976686106,
 'MBE_heavy': -24.1072253280953,
 'PBIAS_heavy': -51.86620233837381,
 'KGE_heavy': 0.6098465908924458,
 'spearman_heavy': 0.3578240412450367,
 'std_obs_heavy': 28.272168034508795,
 'std_model_heavy': 17.428566431530513,
 'r_ext': 0.8475439667478993,
 'NSE_ext': 0.5290087766025937,
 'RMSE_ext': 67.54142959641862,
 'MAE_ext': 57.208165779550626,
 'MBE_ext': -56.65614860289127,
 'PBIAS_ext': -61.45303297923762,
 'KGE_ext': 0.6332581755570368,
 'spearman_ext': 0.31643604983494394,
 'std_obs_ext': 39.68935435534869,
 'std_model_ext': 23.137311643102603,
 'POD': 0.933980533530

In [42]:
a = pd.concat( [pd.DataFrame(RT.CNN_predict  , columns = np.arange(1,5) )  , 
                pd.DataFrame(RT.LSTM_predict  , columns = np.arange(5,9)),
                 pd.DataFrame(obs_train.T[: , 0] , columns = np.arange(9,10) )   ] , axis = 1 )


In [121]:
a

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,10.715675,10.860014,10.329482,10.942718,10.320920,11.257669,10.674845,10.693753,8.976566,9.102497,8.903478,9.281751,9.109915,9.089443,9.294736,9.122534,4.56
1,2.064602,2.052127,1.839955,2.071103,2.128949,2.191948,2.158883,2.260081,3.042701,3.133948,3.027020,3.098053,3.155962,3.124327,2.988570,3.130166,1.44
2,0.643014,0.549645,0.540408,0.682881,0.512714,0.552726,0.651598,0.560183,0.578592,0.640700,0.601865,0.680442,0.659170,0.579627,0.519161,0.632279,0.24
3,0.957492,1.086559,0.863968,1.003580,0.777348,1.020331,0.954301,1.008751,0.285837,0.333453,0.302490,0.338356,0.346675,0.334194,0.261157,0.322470,0.00
4,0.387523,0.448102,0.091860,0.168775,0.288283,0.152217,0.245232,0.169796,0.642888,0.681571,0.706029,0.720275,0.745636,0.677220,0.702172,0.669114,2.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013841,5.874851,5.563263,5.498349,5.766365,5.832624,5.560533,5.877449,5.664510,2.784897,2.817101,2.868936,2.799664,2.887817,2.815983,2.814988,2.758710,0.00
1013842,0.034268,-0.043841,-0.101411,0.172302,0.094026,0.169611,-0.113649,0.082973,0.750534,0.777802,0.844517,0.828339,0.883318,0.771190,0.768823,0.768374,0.00
1013843,0.138089,0.258245,0.243106,0.443902,0.188884,0.307708,0.295468,0.340434,0.378950,0.391833,0.426566,0.429341,0.471756,0.416201,0.420592,0.409142,0.00
1013844,-0.063982,-0.157500,-0.359765,-0.233703,-0.338298,-0.164178,-0.150876,0.002490,0.383864,0.393195,0.390574,0.389351,0.429192,0.391088,0.363451,0.382303,0.00


In [46]:
a[ a[9] >= 60]

,1,2,3,4,5,6,7,8,9
2285,0.873573,1.185136,-0.010502,0.185758,0.345377,1.062876,-0.000833,0.065229,60.24
7300,11.362780,2.476266,-0.049660,0.911273,10.691666,2.336073,0.024948,0.986418,95.28
9038,1.760715,1.398037,-0.020126,0.471982,3.338059,1.620088,0.007078,0.665907,60.48
10630,2.482255,1.267774,0.015154,0.282706,3.994671,1.521889,0.011618,0.459561,73.20
10879,0.334588,1.013936,-0.000815,0.130793,-0.024711,0.988183,-0.005733,-0.026666,62.64
...,...,...,...,...,...,...,...,...,...
1005475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.16
1005993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.72
1006020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.96
1006021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.72


In [50]:
np.max( obs_train.T  , axis = 0)

array([433.44      ,   6.07800521,   1.        ,   1.        ])